### Sentiment Analysis using RNN

In [1]:
# importing the dataset from keras
from keras.datasets import imdb

In [2]:
# loading the dataset
voc_size = 5000

# splitting the dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = voc_size)

/home/asus/.local/lib/python3.7/site-packages/keras/datasets/imdb.py:101: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/home/asus/.local/lib/python3.7/site-packages/keras/datasets/imdb.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [3]:
print(x_train)

[list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32])
 list([1, 194, 1153, 194, 2, 78, 228, 5, 6, 1463, 4369, 2, 134, 26, 

In [4]:
print(y_train)  # labels

[1 0 0 ... 0 1 0]


In [5]:
# maximum review length
print('Maximum review length: {}'.format(len(max((x_train + x_test), key=len))))

MAximum review length: 2697


In [6]:
# minimum review length
print('Minimum review length: {}'.format(len(min((x_train + x_test), key=len))))

Minimum review length: 70


In [7]:
# mapping the id's
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}

print('---review with words---')
print([id2word.get(i, ' ') for i in x_train[2]])
print('---label---')
print(y_train[2])

1646592/1641221 [==============================] - 9s 5us/step
---review with words---
['the', 'as', 'there', 'in', 'at', 'by', 'br', 'of', 'sure', 'many', 'br', 'of', 'and', 'no', 'only', 'women', 'was', 'than', "doesn't", 'as', 'you', 'never', 'of', 'hat', 'night', 'that', 'with', 'ignored', 'they', 'bad', 'out', 'superman', 'plays', 'of', 'how', 'star', 'so', 'stories', 'film', 'comes', 'defense', 'date', 'of', 'wide', 'they', "don't", 'do', 'that', 'had', 'with', 'of', 'hollywood', 'br', 'of', 'my', 'seeing', 'fan', 'this', 'of', 'pop', 'out', 'body', 'shots', 'in', 'having', 'because', 'cause', "it's", 'stick', 'passing', 'first', 'were', 'enjoys', 'for', 'from', 'look', 'seven', 'sense', 'from', 'me', 'and', 'die', 'in', 'character', 'as', 'and', 'issues', 'but', 'is', 'you', 'that', "isn't", 'one', 'song', 'just', 'is', 'him', 'less', 'are', 'strongly', 'not', 'are', 'you', 'that', 'different', 'just', 'even', 'by', 'this', 'of', 'you', 'there', 'is', 'eight', 'when', 'it', 'par

#### pad sequence
    - truncating longer reviews
    - padding shorter reviews with null value
    

In [9]:
from keras.preprocessing import sequence
max_words = 500
x_train = sequence.pad_sequences(x_train, max_words)
x_test = sequence.pad_sequences(x_test, max_words)

## RNN model for Sentiment Analysis

In [12]:
# importing the layers
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# building the model
embedding_size = 32
model = Sequential()
model.add(Embedding(voc_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

# printing the model summary
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________


## Training and evaluation of Model 

In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [15]:
batch_size=64
num_epochs=10 # training only for 10 epochs

x_valid, y_valid = x_train[:batch_size], y_train[:batch_size]
x_train2, y_train2 = x_train[batch_size:], y_train[batch_size:]

model.fit(x_train2, y_train2, validation_data=(x_valid, y_valid), batch_size=batch_size, epochs=num_epochs)


Train on 24936 samples, validate on 64 samples
Epoch 1/10
24936/24936 [==============================] - 132s 5ms/step - loss: 0.4950 - accuracy: 0.7554 - val_loss: 0.4900 - val_accuracy: 0.8125
Epoch 2/10
24936/24936 [==============================] - 123s 5ms/step - loss: 0.3252 - accuracy: 0.8651 - val_loss: 0.2050 - val_accuracy: 0.9219
Epoch 3/10
24936/24936 [==============================] - 134s 5ms/step - loss: 0.2523 - accuracy: 0.9015 - val_loss: 0.1838 - val_accuracy: 0.9219
Epoch 4/10
24936/24936 [==============================] - 198s 8ms/step - loss: 0.2118 - accuracy: 0.9173 - val_loss: 0.1964 - val_accuracy: 0.9062
Epoch 5/10
24936/24936 [==============================] - 123s 5ms/step - loss: 0.2125 - accuracy: 0.9160 - val_loss: 0.3987 - val_accuracy: 0.8750
Epoch 6/10
24936/24936 [==============================] - 144s 6ms/step - loss: 0.1854 - accuracy: 0.9297 - val_loss: 0.2087 - val_accuracy: 0.9062
Epoch 7/10
24936/24936 [==============================] - 130s 5

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', scores[1])